# Simulering av aksjonspotensialet med virrevandring av ioner

Prosjekt 3 i TMA4320 våren 2019 - av Thorvald Ballestad, Jonas Bueie og Herman Sletmoen (gruppe 3)

I denne prosjektoppgaven skal vi simulere aksjonspotensialet i celler, som er raske potensialvariasjoner mellom innsiden og utsiden av celler og som gjerne forplanter seg som et signal langs nerveceller.
Aksjonspotensialet er en konsekvens av ioner som beveger seg inn og ut av celler gjennom cellemembranen i en diffusjonsprosess, påvirket av et elektrisk potensiale.

Vi kommer først til å diskutere noe teori rundt diffusjon og simulere en ren diffusjonsprosess med virrevandring.
Deretter simulerer vi diffusjonsprosessen i et elektrisk potensiale, før vi går løs på å simulere prosessen mellom inn- og utsiden av en cellemembran og til slutt selve aksjonspotensialet.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import constants as consts


# TODO: remove before handing in
try:
    heaviside = np.heaviside
except:
    def heaviside(x1, x2):
        return np.where(x1 < 0, 0, np.where(x1 > 0, 1, x2))

In [ ]:
# Setup plotting parameters
newparams = {'axes.labelsize': 15,
             'axes.linewidth': 1,
             'lines.linewidth': 1.5, 
             'figure.figsize': (16, 8),
             'ytick.labelsize': 15,
             'xtick.labelsize': 15,
             'ytick.major.pad': 5,
             'xtick.major.pad': 5,
             'legend.fontsize': 15,
             'legend.frameon': True, 
             'legend.handlelength': 1.5,
             'axes.titlesize': 20,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)


## Oppgave 2 - diffusjonslikningen

Diffusjonslikningen i én dimensjon uttrykker at den romlige distribusjonen $\phi = \phi(x, t)$ over posisjonen $x$ av en substans som har diffundert i en tid $t$ oppfyller
$$\frac{\partial \phi(x, t)}{\partial t} = \frac{\partial}{\partial x} \left(D(x) \frac{\partial \phi(x, t)}{\partial x}\right),$$
der $D(x)$ er en generelt posisjonsavhengig diffusjonskoeffisient.

### Oppgave 2.1 - eksponensiell løsning

Når $D(x) = D$ er posisjonsuavhengig, reduseres diffusjonslikningen til 
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2},$$
Da er
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
en løsning av diffusjonslikningen, siden
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2} = \left(\frac{(x-x_0)^2}{4Dt^2} - \frac{1}{2t}\right) \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$

### Oppgave 2.2.1 - diffusjon fra initiell punktsamling

Vi skal nå å finne distribusjonen $\phi(x, t)$ ved et vilkårlig tidspunkt for en initiell punktsamling i $x_0$
$$\phi(x, 0) = \delta(x - x_0) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w,$$
der $\delta(x)$ er Diracs deltafunksjon.

#### Med taylorutvikling

Vi antar at distribusjonen i en gitt posisjon $x$ kan taylorutvikles om starttidspunktet $t = 0$ som
$$\phi(x, t) = \sum_{n=0}^{\infty} \frac{t^n}{n!} \frac{\partial^n \phi}{\partial t^n}(x, 0).$$
Den tidsderiverte av første orden ved et vilkårlig tidspunkt er gitt direkte ved diffusjonslikningen som
$$\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi(x, t)}{\partial x^2}.$$
Ved å utnytte diffusjonslikningen, og å bytte derivasjonsrekkefølge, finner vi den tidsderiverte av andre orden
$$\frac{\partial^2 \phi(x, t)}{\partial t^2} = \frac{\partial}{\partial t} D \frac{\partial^2 \phi(x, t)}{\partial x^2} = D \frac{\partial^2}{\partial x^2}\frac{\partial \phi(x, t)}{\partial t} = D^2 \frac{\partial^4 \phi(x, t)}{\partial x^4}.$$
Argumentet kan gjentas flere ganger, og vi får generelt
$$\frac{\partial^n \phi(x, t)}{\partial t^n} = D^n \frac{\partial^{2 n} \phi(x, t)}{\partial x^{2 n}}.$$

Ved $t = 0$ får vi
$$\frac{\partial^n \phi}{\partial t^n}(x, 0) = D^n \frac{\partial^{2 n}}{\partial x^{2 n}} \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-i w (x - x_0)} \mathrm{d} w = D^n \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \frac{\partial^{2 n}}{\partial x^{2 n}} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w.$$
Ved innsetting av dette i taylorrekken, får vi at 
$$\phi(x, t) = \frac{1}{2 \pi} \sum_{n=0}^{\infty} \frac{t^n}{n!} \int_{-\infty}^{+\infty} (-Dw^2)^{n} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \sum_{n=0}^{\infty} \frac{(-D w^2 t)^n}{n!} e^{-i w (x - x_0)} \mathrm{d} w = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-D w^2 t} e^{-i w (x - x_0)} \mathrm{d} w.$$
der vi har byttet rekkefølge på integrasjonen og summasjonen, og gjenkjent taylorrekken til eksponensialfunksjonen.
Det siste integralet kan skrives som
$$\phi(x, t) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} e^{-(w \sqrt{D t} + \frac{i(x-x_0)}{2\sqrt{Dt}})^2} e^{-\frac{(x-x_0)^2}{4Dt}} \mathrm{d}w.$$

Om vi innfører $z = w \sqrt{Dt} + \frac{i(x-x_0)}{2\sqrt{Dt}}$, slik at $\mathrm{d}w = \mathrm{d}z/\sqrt{Dt}$, får vi
$$\phi(x, t) = \frac{1}{2 \pi \sqrt{Dt}} e^{-\frac{(x-x_0)^2}{4Dt}} \int_{-\infty}^{+\infty} e^{-z^2} \mathrm{d}z.$$
Integralet i denne likningen har verdien $\sqrt{\pi}$, og vi får til slutt
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}.$$

#### Med fourieranalyse (alternativ løsningsmetode)

Med Fourieranalyse kan løsningen finnes med betydelig mindre arbeid ved å innføre en Fouriertransformasjon $\hat{\phi} = \hat{\phi}(w, t)$ av $\phi$ med hensyn kun på posisjonen $x$.
Siden $\widehat{\partial \phi / \partial t} = \partial \hat{\phi} / \partial t$ og $\widehat{\partial^2 \phi / \partial x^2} = (iw)^2 \hat{\phi}$, må fouriertransformasjonen oppfylle
$$\frac{\partial \hat{\phi}(w, t)}{\partial t} = -Dw^2 \hat{\phi}(w, t),$$
hvilket må bety at
$$\hat{\phi}(w, t) = Ae^{-Dw^2t},$$
for en konstant $A$.
Distribusjonen $\phi$ må da være gitt ved den inverse fouriertransformasjonen
$$\phi(x, t) = \frac{A}{\sqrt{2 \pi}} \int_{-\infty}^{+\infty} e^{-Dw^2t}e^{iwx} \mathrm{d}w,$$
som oppfyller initialbetingelsen dersom vi velger $A = 1/\sqrt{2 \pi}$ og gjør koordinatskiftet $x \rightarrow x - x_0$.
Vi kan så finne $\phi$ ved å beregne integralet på samme måte som før, eller simpelthen ved å slå opp den inverse fouriertransformasjonen av $\hat{\phi}$ i en tabell.

### Oppgave 2.2.2 - fysisk tolkning av diffusjonskoeffisienten

Med $\sigma^2 = 2 D t$, er løsningen 
$$\phi = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x - x_0)^2}{2 \sigma^2}},$$
som tilsvarer sannsynlighetsfordelingen til en normalfordeling med forventningsverdi $x_0$ og varians $\sigma^2$.
Siden variansen vokser lineært med tiden og beskriver spredningen til fordelingen, er diffusjonskoeffisienten $D$ et mål på hvor raskt substansen diffunderer.

### Oppgave 2.2.3 - diffusjon fra vilkårlig initiell spredning 

Diffusjonslikningen er lineær, slik at en lineær kombinasjon av flere løsninger også løser likningen.
Vi kan, ved hjelp av den fundamentale løsningen vi fant i oppgave 2.2.1, utnytte denne egenskapen til å uttrykke distribusjonen ved et vilkårlig tidspunkt $t$, gitt en initiell fordeling $\phi(x, 0) = f(x).$

Dersom
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} e^{-\frac{(x - x_0)^2}{4 D t}}$$
løser diffusjonslikningen med $\phi(x, 0) = \delta(x - x_0),$ vil lineærkombinasjonen
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \sum_{n=-\infty}^{+\infty} c_n e^{-\frac{(x - x_n)^2}{4 D t}}$$
være en løsning med initialbetingelsen $\phi(x, 0) = \sum_{n=-\infty}^{+\infty} c_n \delta(x - x_n),$ og integralet
$$\phi(x, t) = \frac{1}{\sqrt{4 \pi D t}} \int_{-\infty}^{+\infty} f(y) e^{-\frac{(x - y)^2}{4 D t}} \mathrm{d}y$$
vil være en løsning med en generell initiell distribusjon $\phi(x, 0) = \int_{-\infty}^{+\infty} g(y) \delta(x - y) \mathrm{d}y = f(x).$
Summasjonstegnet er enkelt og greit erstattet med et integral, mens $f(y) \mathrm{d}y$ spiller rollen til konstantene $c_n$.

## Oppgave 3 - virrevandring uten potensiale

I denne oppgaven lar vi 1000 partikler, som opprinnelig befinner seg samlet i origo, gjennomgå en virrevandringsprosess over 100 tidssteg.
Ved hvert tidssteg hopper hver partikkel enten til høyre eller venstre med like stor sannsynlighet.
Til slutt viser vi posisjonsfordelingen til alle partiklene i et histogram, og sammenlikner fordelingen med en normalkurve tilpasset dataene.

In [ ]:
N = 1000
n_steps = 100

plt.title("Distribution of %d initially gathered particles after %d random walk steps" % (N, n_steps))
plt.xlabel("Position")
plt.ylabel("Proportion of particles")
    
# simulate random walk
pos = np.zeros(N)
steps = np.random.randint(0, 2, size=(n_steps, N))*2 - 1 # Generate random sequence of +1 and -1
step  = np.sum(steps, axis=0)
pos += step
plt.hist(pos, normed = True, label="simulated")

# fit normal curve
mean, stddev = norm.fit(pos)
x = np.linspace(*plt.xlim(), 100)
p = norm.pdf(x, mean, stddev)
plt.plot(x, p, label="theoretical")

plt.legend()
plt.show()

Fra teorien i seksjon 1 i oppgaveteksten vet vi at virrevandring med infinitesimale tids- og posisjonssteg svarer til diffusjon, som beskrevet av diffusjonslikningen $^{[1]}$.
Videre er det vist i oppgave 2 at den forventede posisjonsfordelingen til en samling partikler som diffunderer fra samme utgangspunkt, etter en tid $t$ er normalfordelt om startposisjonen, med en varians proporsjonal med $t$.
Vi ser av figuren at den simulerte partikkelfordelingen og den tilpassede normalkurven samsvarer godt med hverandre og denne teorien.

## Oppgave 5 - virrevandring i tidsuavhengig potensiale

I denne oppgaven simulerer vi en tilsvarende virrevandringsprosess som i oppgave 3, men her i et potensiale der sannsynligheten for å hoppe til venstre eller høyre fra et punkt avhenger av potensialet rundt punktet.
Ved hjelp av statistisk mekanikk kan vises at sannsynligheten for at en partikkel beveger seg til høyre fra posisjonen $x$ i en slik virrevandringsprosess er
$$P^+ = \frac{1}{1 + e^{-\beta (V(x-h)-V(x+h))}},$$
der $\beta = 1 / k_B T$ er invers termisk energi og $V(x \mp h)$ er partikkelens potensielle energi i nabopunktene.
Sannsynligheten for å hoppe til venstre er da
$$P^- = 1 - P^+.$$
Simuleringen vil gjøres med flere ulike potensialer, men ellers samme parametere og initialbetingelse som i oppgave 3.

### Oppgave 5.1 - lineært potensiale

Først simulerer vi virrevandringen til partikler med lineær potensiell energi
$$V(x) = k x,$$
for en konstant $k$, som representerer en konstant kraft på partiklene.

In [ ]:
def P_right(x, V, betak = 1):
    ratio = np.exp(-betak * (V(x-1) - V(x+1)))
    return 1/(1 + ratio)

def simulate(pos, V, n_steps, betak = 1):
    pos = np.copy(pos) # Do not change the input array
    x = np.arange(-n_steps, +n_steps + 1) # all possible positions in simulation
    
    for t in range(n_steps): # Go through time
        prob_right_by_pos = P_right(x, V, betak)
        prob_right_by_ion = prob_right_by_pos[pos - x[0]]
        
        step = np.where(np.random.rand(len(pos)) <= prob_right_by_ion, +1, -1)
        pos += step
    
    return pos

def scale_vals_into_range(rmin, rmax, vals):
    vmin = np.min(vals)
    vmax = np.max(vals)
    return rmin + (vals - vmin) / (vmax - vmin) * (rmax - rmin)

def simulate_multiple_kbetas(V, kbetas, N=1000, n_steps=200, potential_desc=None):
    title_base = "Distribution of %d particles after %d random walk steps from origin" % (N, n_steps)
    if potential_desc is not None:
        title_base += " in %s potential" % potential_desc
        
    for kbeta in kbetas:
        pos = np.full(N, 0)
        pos = simulate(pos, V, n_steps, kbeta)

        title = title_base + " ($k \\beta = %r$)" % kbeta
        plt.title(title)
        plt.xlabel("Position")
        plt.ylabel("Proportion of particles")
        plt.xlim(-n_steps, +n_steps) # common for all plots
        
        x = np.arange(-n_steps, +n_steps + 1)
        bins = np.arange(-n_steps, +n_steps + 1, 2) # bin width 2 to cover alternating holes
        fracs, _, _ = plt.hist(pos, bins=bins, normed=True, label="distribution")
        fmax = np.max(fracs)
        v = V(x)
        v = scale_vals_into_range(0, 0.5 * fmax, v)
        plt.plot(x, v, label="potential (qualitative)") # plot potential qualitatively
        
        plt.legend()
        plt.show()

### Oppgave 5.1 - lineært potensiale

Først simulerer vi virrevandringen i det lineære potensialet
$$V(x) = k x$$
for en konstant $k$, som representerer en konstant kraft på partiklene.

In [ ]:
def V_linear(x):
    return x

simulate_multiple_kbetas(V_linear, (0.5, 1.0, 1.5), potential_desc="linear")

Figurene viser fordelingen av partikler før og etter diffusjonsprosessen, samt potensialfunksjonen, for tre ulike verdier av $k\beta$. 
Figuren viser posisjonsfordelingene for $k\beta = 1$ og $k\beta = 0$, samt den potensielle energien som funksjon av posisjon. Det fremkommer av denne og de tidligere figurene at partiklene vandrer lengre når termisk energi er liten. Når dette er tilfelle, er sannsynligheten for at en partikkel skal gå oppover potensialgradienten liten i ethvert punkt, slik at partiklene forflytter seg lengre i retning lavt potensial.

### Oppgave 5.2 - bokspotensiale

Vi simulerer nå virrevandringsprosessen til partikler med potensiell energi
$$V(x) = \begin{cases} k & -3h < x < +3h \\ 0 & \text{ellers} \\ \end{cases}.$$
Disse partikklene beveger seg uten påvirkning av krefter der potensialet er flatt, og utsettes for en krefter i overgangsområdene mellom disse flate regionene, når $x = \pm 3h$.

In [ ]:
def V_membrane(x, d = 3):
    return np.where(np.logical_and(-d < x, x < d), 1, 0)

simulate_multiple_kbetas(V_membrane, (0.5, 1.0, 1.5), potential_desc="box")

Figuren viser posisjonsfordelingen etter at diffusjonsprosessen er gjennomført, sammen med den tilhørende potensielle energifunksjonen. I alle posisjoner der den potensielle enrgien er flat, virker det ikke krefter på partiklene.

### Oppgave 5.3 - lineært potensiale med flate endestykker

Her simulerer vi virrevandringsprosessen til partikler med potensiell energi
$$V(x) = \begin{cases} -k & x \leq -3h \\ k \left(-1 + 2 \frac{x+3h}{6h} \right) & -3h < x < +3h \\ +k & x \geq +3h \\ \end{cases}.$$
Disse partiklene påvirkes av en konstant kraft i et avgrenset område (mellom $-3h$ og $3h$), men er ellers upåvirket.

In [ ]:
def V_linear_edge(x):
    # Alternatively reuse V_membrane here
    return np.where(np.logical_and(-3 < x, x < 3), -1 + 2*(x + 3)/6, np.sign(x))

simulate_multiple_kbetas(V_linear_edge, (0.5, 1.0, 1.5), potential_desc="restricted linear")

Figuren viser resultatet avv diffusjonsprosessen for nevnte potensielle energifunksjon, for to ulike verdier av $k\beta$.

## Oppgave 7 og 8 - virrevandring av ioner i cellemembran med tidsavhengig potensiale og simulering av aksjonspotensialet

In [ ]:
T = 273 + 37 # K, body temperature
beta = 1 / (consts.k*T)
C = 0.07e3 # e mM / V

gate_lower_v_limit = -70e-3 # Volts
gate_upper_v_limit = +30e-3 # Volts

V_beta_closed = 50
V_closed = V_beta_closed / beta # closed gate potential for both Na and K

def inside_concentration(pos):
    cons_scaling_factor = 0.1 # mM, the concentration each particle represents
    n_inside = len(pos[pos > 0])
    return n_inside * cons_scaling_factor

def outside_concentration(pos):
    return inside_concentration(-pos)

def V_diff(pos_Na, pos_K, conc_out, C):
    conc_in = inside_concentration(pos_Na) + inside_concentration(pos_K)
    conc_diff = conc_in - conc_out
    return conc_diff / C

def simulate_v2(V0_Na_beta=1, V0_K_beta=1, n_steps=1000, L=50, action_potential=False, pump=False):
    V0_Na = V0_Na_beta / beta
    V0_K = V0_K_beta / beta

    # Put ions in start positions (negative positions outside, positive positions inside)
    N_Na_inside = 50
    N_Na_outside = 1450
    N_K_inside = 1400
    N_K_outside = 50
    N_Na = N_Na_inside + N_Na_outside
    N_K = N_K_inside + N_K_outside
    pos_Na = np.append(np.full(N_Na_outside, -L//4), np.full(N_Na_inside, +L//4))
    pos_K = np.append(np.full(N_K_outside, -L//4), np.full(N_K_inside, +L//4))
    
    conc_out = outside_concentration(pos_Na) + outside_concentration(pos_K) # constant throughout sim
      
    if action_potential:
        # Start with gates closed
        V_Na = V_closed
        V_K = V_closed
    else:
        V_Na = V0_Na
        V_K = V0_K
    
    if pump:
        n_Na_per_pumping = 3
        n_K_per_pumping = 2
        steps_per_pumping = 10
        
    x = np.arange(-L//2, +L//2 + 1) # all possible positions in simulation

    v_diffs = []

    for t in range(n_steps): # Go through time
        v_diff = V_diff(pos_Na, pos_K, conc_out, C)
        v_diffs.append(v_diff)
        v_diff_array = v_diff * heaviside(x, 0.5) * consts.e
        membrane = V_membrane(x, d=1)
        
        if action_potential:
            if v_diff < gate_lower_v_limit:
                V_Na = V0_Na # open sodium gate
                V_K = V_closed # close potassium gate
            elif v_diff > gate_upper_v_limit:
                V_Na = V_closed # close sodium gate
                V_K = V0_K # open potassium gate
        
        for V, pos in zip((V_Na, V_K), (pos_Na, pos_K)):
            v_total = v_diff_array + V * membrane # total potential experienced by ion
            prob_right_by_pos = 1 / (1 + np.exp(beta * (np.roll(v_total, -1) - np.roll(v_total, +1))))
            prob_right_by_pos[-1] = 0 # force rightmost ions to jump left
            prob_right_by_pos[0] = 1 # force leftmost ions to jump right
            prob_right_by_ion = prob_right_by_pos[pos - x[0]]

            # jump left/right with calculated probabilities
            step = np.where(np.random.rand(len(pos)) <= prob_right_by_ion, +1, -1)
            pos += step

        if pump and t % steps_per_pumping == 0:
            pos_Na[::+1].sort() # sort ascending
            pos_K[::-1].sort() # sort descending
            index_Na = np.argwhere(pos_Na > 0)[:n_Na_per_pumping] # pump ions closest to membrane
            index_K = np.argwhere(pos_K < 0)[:n_K_per_pumping] # pump ions closest to membrane
            pos_Na[index_Na] = -1 # pump Na ions out, place them just outside
            pos_K[index_K] = +1 # pump K ions in, place them just inside
                
    return pos_Na, pos_K, v_diffs

### Oppgave 7.1 og 7.2 - virrevandring i cellemembran med identiske og ulike ionekanalpotensialer

In [ ]:
plt.title("Membrane voltage in random walk around cell membrane without action potential and pump")
plt.xlabel("Random walk steps")
plt.ylabel("Membrane voltage (V)")

for V0_Na_beta, V0_K_beta in ((1,1), (10,1), (1,10)):
    _, _, v_diffs = simulate_v2(V0_Na_beta, V0_K_beta)
    label = "$\\beta V_{Na}^0 = %r ,\\quad \\beta V_{K}^0 = %r$" % (V0_Na_beta, V0_K_beta)
    plt.plot(v_diffs, label=label)
    
plt.legend(loc="upper right")
plt.show()

### Oppgave 8.1 - aksjonspotensialet uten ionepumpe

In [ ]:
plt.title("Membrane voltage in random walk around cell membrane with action potential and without pump")
plt.xlabel("Random walk steps")
plt.ylabel("Membrane voltage (V)")

pos_Na, pos_K, v_diffs = simulate_v2(action_potential=True, pump=False)
plt.plot(v_diffs)
plt.axhline(gate_upper_v_limit, color="red", linestyle="dashed", label="upper gate voltage limit")
plt.axhline(gate_lower_v_limit, color="red", linestyle="dashed", label="lower gate voltage limit")

plt.legend(loc="upper right")
plt.show()

### Oppgave 8.2 - aksjonspotensialet med ionepumpe

In [ ]:
plt.title("Membrane voltage in random walk around cell membrane with action potential and pump")
plt.xlabel("Random walk steps")
plt.ylabel("Membrane voltage (V)")

pos_Na, pos_K, v_diffs = simulate_v2(action_potential=True, pump=True)
plt.plot(v_diffs)
plt.axhline(gate_upper_v_limit, color="red", linestyle="dashed", label="upper gate voltage limit")
plt.axhline(gate_lower_v_limit, color="red", linestyle="dashed", label="lower gate voltage limit")

plt.legend(loc="upper right")
plt.show()

## Referanser
[1] Simensen, Håkon T: *Prosjekt 3 - TMA4320 - Simulating the Action Potential with Random Walk of Ions.*